# Графовый анализ тем ВКР на программе Международные отношения в 2024 году.
media and press

RPC and China

Catalan separatist movement and Catalan independence movement

In [2]:
from graph import MyGraph
g = MyGraph("2024")
dct = g.process_final_json(to_save=True)